# Campus Job Notifier

Most people don’t know when the new jobs  will post in our student employment website. So people, who are looking for jobs on the campus  they need to check the website regularly. The objective of this application  is to reduce this work and automate tasks and send  a notification to the user about jobs.

## Procedure
1. Web Scrape university student opportunities website to get the jobs list and html links to get the detailed information on each job.
2. Compare the present job details with previous job details and get the details of the  new job.
3. Automatically generate the Pie chart by using the present job details list.
4. Send Notifications to their Emails about new jobs posted in the campus student employment website. 


## Libraries Required:
1. BeautifulSoup:
Beautiful Soup is a library that makes it easy to scrape information from web pages. It sits atop an HTML or XML parser, providing Pythonic idioms for iterating, searching, and modifying the parse tree.

2. Requests:
Requests is a Python module you can use to send all kinds of HTTP requests.

3. Simple Mail Transfer Protocol(SMTP):
Simple Mail Transfer Protocol (SMTP) is a protocol, which handles sending e-mail and routing e-mail between mail servers

4. Pickle:
Python pickle module is used for serializing and de-serializing a Python object structure. Any object in Python can be pickled so that it can be saved on disk.

5. Counter:
A Counter is a container that keeps track of how many times equivalent values are added. It can be used to implement the same algorithms for which bag or multiset data structures are commonly used in other languages. It is from collections library.

6. Pylab:
PyLab combines the numerical module numpy with the graphical plotting module pyplot. 

7. Matplotlib: 
Matplotlib is a Python 2D plotting library which produces publication quality figures in a variety of hardcopy formats and interactive environments across platforms.

8. email.mime:
The email package library for managing email messages. This library used to send email messages to SMTP.


Importing Libraries:

In [1]:
from bs4 import BeautifulSoup
import requests,smtplib
import pickle
from collections import Counter
from pylab import rcParams
import matplotlib.pyplot as plt
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
# It makes the make your requests as legitimate
headers = ({'User-Agent':'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36'})

This function will get all the jobs URL's of each job to get all the jobs details.

In [2]:
def job_links():
    list =[]
   
    url = "https://agency.governmentjobs.com/nwmissouri/default.cfm?promotionaljobs=1"
    get_page = requests.get(url, headers=headers)
    Page = BeautifulSoup(get_page.text, 'html.parser')
    #print(Page)
    list = Page.find_all('a',class_="jobtitle")
    links = []
    count = 0   
    for job in list:
        try:
        
            page_url ="https://agency.governmentjobs.com/nwmissouri/"+job['href']
            #print(page_url)
            count = count + 1;
            links.append(page_url)
        except:
                pass
    return links;

This function get the all the jobs details and compares current jobs with previous jobs. It returns new job details and dump the current job details in the pickle in place of previous job details.

In [1]:
def job_details(links):
 
    old_job_titles = []
    fresh_job_titles = []
    fresh_job_departments = []
    list_job_details = {
               "data":[]
                }
    for url in links:
        headers = ({'User-Agent':'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36'})
        get_page = requests.get(url, headers=headers)
        individual_job_page = BeautifulSoup(get_page.text, 'html.parser')
        data = individual_job_page.find_all('table')
        
        details=[]
        for d in data[1].findAll('td'):      
            details.append((d.text).strip())
            
        job_details = {
                "Job Title": details[0],
                "Opening Date/Time": details[1],
                "Salary": details[2],
                "Job Type": details[3],
                "Location": details[4],
                "Department": details[5]
                }
        list_job_details['data'].append(job_details)
    

    old_job_titles = job_list();
    
    for d in list_job_details['data']:
        title = d['Job Title'] 
        department = d['Department']
        if(not old_job_titles.__contains__(title)):
            fresh_job_titles.append(title)
            fresh_job_departments.append(department)
            

    
    with open('jobs.pkl', 'wb') as f:
        pickle.dump(list_job_details,f,protocol=pickle.HIGHEST_PROTOCOL) 
    return(fresh_job_titles,fresh_job_departments)

This function lists the previous jobs titles from jobs.pkl file. This list is used in the job_details function for comparison between the current job details and previous job details.

In [ ]:
def job_list():
    jobs_titles = []
    prev_list_job_details = {
               
                }
    with open('jobs.pkl', 'rb') as f:        
        sentences = pickle.load(f)
        prev_list_job_details = sentences
    if(len(prev_list_job_details)!=0):        
        for l in prev_list_job_details['data']:
            title = l['Job Title']
            jobs_titles.append(title)
    return jobs_titles

This function plots the pie chart between the titles and departments.
This shows "How many jobs per department."

In [ ]:
def comparission():        
        titles, departments = job_details(links)
# Counter is a container that keeps track of how many times equivalent values are added and returns the dictionary
        a = dict(Counter(departments))
        def getKeys(a): 
            list_k = [] 
            for key in a.keys(): 
                list_k.append(key) 
                
            return list_k
            
        def getValues(a): 
            list_v = [] 
            for value in a.values(): 
                list_v.append(value) 
                    
            return list_v
        keys = getKeys(a)
        values = getValues(a)
        matplotlib.rcParams.update({'font.size': 15})       
        rcParams['figure.figsize'] = 11, 10
        print(keys)
        print(values)
        plt.title('How many jobs per department')
        fig = plt.figure()
        plt.pie(values, labels=keys, startangle=90, autopct='%1.1f%%')
        plt.axis('equal')  
        plt.tight_layout()
        plt.show()
        fig.savefig('plot.png')  

This function sends notifications by using the SMTP and email.meme libraries. I have used HTML code to represent the mail proper manner.

In [ ]:
def notification():    
    titles, departments = job_details(links)
    mailbody = ""
    for title in titles:
        mailbody += title +"\n"
    a = dict(Counter(departments))
    def getKeys(a): 
        list_k = [] 
        for key in a.keys(): 
            list_k.append(key) 
                
        return list_k
            
    def getValues(a): 
        list_v = [] 
        for value in a.values(): 
            list_v.append(value) 
                    
        return list_v
    keys = getKeys(a)
    values = getValues(a)
    
    table_0 = """</br></br></br><h2>Number of jobs per department</h2>
    <tr><th>Departments </th><th>&nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; Number of Jobs Per Department</th></tr>"""
    table_1 = """</br></br></br></table>"""
    table_data = ""
    count = 0
    for d in keys:
        table_data += "<tr><td>"+d+ "</td>"+"<td>"+ str(values[int(count)]) + "</td></tr>"
        count = count + 1
    
    for title in titles:
        mailbody += "<h5>"+title +"</h5>" +"\n"
  
    
    sender_email = "Enter Sender Email Id"
    receiver_email = "Enter Receiver Email Id"
    password = "Enter Sender Email Password"
    
    message = MIMEMultipart("alternative")
    message["Subject"] = "multipart test"
    message["From"] = sender_email
    message["To"] = receiver_email
    
    
    # Create the plain-text and HTML version of your message
   
    
    html_0 = """\
        <html>
        <body>
        <h1 style= font-size= "25px", align="center">Jobs updated in the Student Employment Opportunities Page</h1>
        <h2>List of jobs<br></h2>
        """
        
    html_1 = """\ 
        </body>
        </html>
        """
    
    
    if(len(titles)!=0):
        text = """\
         Jobs updated in the Student Employment Opportunities Page
        """
        html = html_0 + mailbody+"</br></br></br>"+ table_0 + table_1+ table_data + html_1
        
    else:
        text = """\
        Jobs does not updated
        """
        html = """\
        <html>
        <body>
        <h1 style= bgcolor="black", font-size= "25px", align="center">Jobs does not updated in the Student Employment Opportunities Page</h1>
        </body>
        </html>
        """
    # Turn these into plain/html MIMEText objects
    part1 = MIMEText(text, "plain")
    part2 = MIMEText(html, "html")

    # Add HTML/plain-text parts to MIMEMultipart message
    # The email client will try to render the last part first
    message.attach(part1)
    message.attach(part2)

    with smtplib.SMTP_SSL("smtp.gmail.com", 465) as server:
        server.login(sender_email, password)
        server.sendmail(
                sender_email, receiver_email, message.as_string()
                )

In [ ]:
# calling all the functions
links = job_links()
job_details(links)
notification()
comparission()

Mail Screenshot when the jobs are updated

![Jobs Updated](Jobs-Updated.PNG)

Mail Screenshot when the Jobs are not updated 

![Jobs Not Updated](Jobs_not_updated.PNG)

## Conclusion

This application provides the solution to the people looking for new jobs regularly. They get the notifications and reduce their valuable amount of time by running application regularly it automates all the tasks.
